In [1]:
import pandas as pd
import random
from pydataquery import DataQuery
from matplotlib.colors import LinearSegmentedColormap
from xbbg import blp
import numpy as np
import pytz
import time
from datetime import datetime, timedelta
import statsmodels.api as sm
from ipywidgets import interact, Dropdown, HBox, VBox, Button, Output, Text, widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import re
import scipy.stats as stats
from adjustText import adjust_text
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sympy as sp
import itertools
import warnings
warnings.filterwarnings("ignore")

dft = pd.read_excel("DQ_Data_3.xlsx")
dft.set_index(dft.columns[0],inplace=True)
dft.index.name = None
dft.index = pd.to_datetime(dft.index, format='%Y-%m-%d').date
df_back = dft.copy()

start_date = df_back.index.max()

##################  Change here only
isin_list = ["XS2214237807", "XS2214239506", "XS2214238441", "XS2214239175", "US040114HX11", "US040114HS26", "US040114HT09", "US040114HU71", "US040114HV54", "US040114HW38", "XS1910826996", "XS1717011982", "XS2384698994", "XS2445169985", "XS1777972511", "XS1910827887", "XS2948511949", "XS1566179039", "XS2384701020", "XS2948512913", "XS1777972941", "XS1717013095", "XS1910828182", "XS2384704800", "XS1843435840", "XS1781710543", "XS2764839945", "XS1843435766", "XS2354781614", "XS1781710626", "XS1318576086", "XS1819680288", "XS2083302419", "XS2446175577", "XS1819680528", "XS2083302500", "XS1790104530", "XS1619155564", "XS2333676133", "XS1790134362", "XS2297220423", "XS1903488572", "XS1558078736", "XS2391394348", "XS1775618439", "XS1504948776", "XS1953057061", "XS1903489463", "XS2297226545", "XS2079842642", "XS2176897754", "XS2391395154", "XS1558078496", "XS1775617464", "XS1953057491", "XS2176899701", "XS2391398174"]
names_list = ["ECUA 6.9 07/31/30", "ECUA 0 07/31/30", "ECUA 5 1/2 07/31/35", "ECUA 5 07/31/40", "ARGENT 1 07/09/29", "ARGENT 0 3/4 07/09/30", "ARGENT 4 1/8 07/09/35", "ARGENT 5 01/09/38", "ARGENT 3 1/2 07/09/41", "ARGENT 4 1/8 07/09/46", "NGERIA 7 5/8 11/21/25", "NGERIA 6 1/2 11/28/27", "NGERIA 6 1/8 09/28/28", "NGERIA 8 3/8 03/24/29", "NGERIA 7.143 02/23/30", "NGERIA 8.747 01/21/31", "NGERIA 9 5/8 06/09/31", "NGERIA 7 7/8 02/16/32", "NGERIA 7 3/8 09/28/33", "NGERIA 10 3/8 12/09/34", "NGERIA 7.696 02/23/38", "NGERIA 7 5/8 11/28/47", "NGERIA 9.248 01/21/49", "NGERIA 8 1/4 09/28/51", "KENINT 7 05/22/27", "KENINT 7 1/4 02/28/28", "KENINT 9 3/4 02/16/31", "KENINT 8 05/22/32", "KENINT 6.3 01/23/34", "KENINT 8 1/4 02/28/48", "ANGOL 9 1/2 11/12/25", "ANGOL 8 1/4 05/09/28", "ANGOL 8 11/26/29", "ANGOL 8 3/4 04/14/32", "ANGOL 9 3/8 05/08/48", "ANGOL 9 1/8 11/26/49", "SENEGL 4 3/4 03/13/28", "SENEGL 6 1/4 05/23/33", "SENEGL 5 3/8 06/08/37", "SENEGL 6 3/4 03/13/48", "EGYPT 3 7/8 02/16/26", "EGYPT 7 1/8 11/10/26", "EGYPT 7 1/2 01/31/27", "EGYPT 5.8 09/30/27", "EGYPT 6.588 02/21/28", "EGYPT 7 11/10/28", "EGYPT 7.6003 03/01/29", "EGYPT 7 5/8 11/10/30", "EGYPT 5 7/8 02/16/31", "EGYPT 7.0529 01/15/32", "EGYPT 7 5/8 05/29/32", "EGYPT 7.3 09/30/33", "EGYPT 8 1/2 01/31/47", "EGYPT 7.903 02/21/48", "EGYPT 8.7002 03/01/49", "EGYPT 8 7/8 05/29/50", "EGYPT 8 3/4 09/30/51"]

#Making Manual Adj. for Perp Bond of PEMEX
# names_list[names_list.index("PEMEX 6 5/8 PERP")] = "PEMEX 6 5/8 01/01/9999"
##################  Change here only

names_list = [item.split(' ',1)[0] + ' ' + item.rsplit(' ',1)[0].split(' ',1)[1] + '% due ' + item.rsplit('/',1)[-1] for item in names_list]

fields = [
          ["DB(SAGE,INSTRUMENT,id_isin,", ",StatCdsPESpreadAsk)"],
          ["DB(SAGE,INSTRUMENT,id_isin,", ",StatCdsPESpreadBid)"],
          ["DB(SAGE,INSTRUMENT,id_isin,", ",StatMidPrice)"],         
          ["DB(SAGE,INSTRUMENT,id_isin,", ",AM_IDX_TOT)"],
          ["DB(SAGE,INSTRUMENT,id_isin,", ",StatRtAskStripSolMid)*100"],
         ]
          
field_names = [
               'PECS Ask', 'PECS Bid', 'Price', 'TR', 'ZSprd',
]

dq_list = []
full_names = []

for i in range(len(isin_list)):
    for j in range(len(fields)):
        dq_list = dq_list + [fields[j][0] + isin_list[i] + fields[j][1]]
        full_names = full_names + [names_list[i] + ' ' + field_names[j]]

labels = dict(zip(full_names, dq_list))

dq = DataQuery(
    client_id='jbAIMF2Tkp0JO3sc',
    client_secret='d7qfzgt55pddjs352sgxosFyI4t2eja07k7opbi6wg9oqjc1OjkdAksn1btmnugeMjchcx2vwTsJupw',
)

job = dq.create_job(expressions = list(labels.values()))
dq.start_date = str(start_date)
job.execute()
df = job.to_pivot_table()
df = df.T
df.index.name = 'Date'
df.index = pd.to_datetime(df.index, format='%Y%m%d').date
df.columns.name = None

df1 = pd.DataFrame()
for key in labels:
    df1[key] = df[labels[key]]

df1 = df1[list(labels.keys())].copy()
clear_output(wait=False)
df1.dropna(axis=1, how='all', inplace=True)

df = df1.copy()

if not (df_back.tail(1).round(4).equals(df.head(1).round(4))):
    print('Check data and re-run')
    dq = DataQuery(
    client_id='jbAIMF2Tkp0JO3sc',
    client_secret='d7qfzgt55pddjs352sgxosFyI4t2eja07k7opbi6wg9oqjc1OjkdAksn1btmnugeMjchcx2vwTsJupw',
    )
    job = dq.create_job(expressions = list(labels.values()))
    dq.start_date = 'TODAY-2Y'
    job.execute()
    df = job.to_pivot_table()
    df = df.T
    df.index.name = 'Date'
    df.index = pd.to_datetime(df.index, format='%Y%m%d').date
    df.columns.name = None
    
    df1 = pd.DataFrame()
    for key in labels:
        df1[key] = df[labels[key]]
    
    df1 = df1[list(labels.keys())].copy()
    clear_output(wait=False)
    df1.dropna(axis=1, how='all', inplace=True)
    df_back = df1.copy()
    df_back.to_excel("DQ_Data_3.xlsx")

else:
    df_back = pd.concat([df_back[~df_back.index.isin(df.index)], df])
    df_back.to_excel("DQ_Data_3.xlsx")

df1 = df_back.copy()
df2 = df1.copy()

for y in [item.rsplit(' ',2)[0] for item in df2.columns.tolist() if 'PECS Ask' in item]:
    df2[f'{y} PECS'] = 0.5 *(df2[f'{y} PECS Bid'] + df2[f'{y} PECS Ask'])

df2.columns = df2.columns.str.replace("PECS","Pecs")
df2.columns = df2.columns.str.replace("ZSprd","Z-sprd")

for y in [item.rsplit(' ',1)[0] for item in df2.columns.tolist() if 'TR' in item]:
    df2[f'{y} FWD_2M_Rtn'] = df2[f'{y} TR'].pct_change(-42)*100
    df2[f'{y} FWD_6M_Rtn'] = df2[f'{y} TR'].pct_change(-126)*100

df2.drop([col for col in df2.columns if (('Ask' in col) or ('Bid' in col) or ('TR' in col) \
                             or ('Duration' in col))], axis=1, inplace=True)

df2 = df2[[col for col in df2.columns if not (('ECUA' in col) and ('Pecs' in col))]]


df = df2.copy()

options = df.columns.to_list()
options = sorted(options, key=lambda item: 2000 + eval(item.rsplit(' ', 2)[-2]) \
                 if eval(item.rsplit(' ', 2)[-2]) < 2000 else eval(item.rsplit(' ', 2)[-2]))

fwd = [item for item in options if 'FWD_' in item]
non_fwd = [item for item in options if not('FWD_' in item)]
options = non_fwd + fwd

issuer = list(sorted(set([item.split(' ',1)[0] for item in options])))

time = ['All','2Y','1Y','6M','3M']

options1 = list(sorted(set([item.rsplit(' ',1)[0] for item in options])))

df1 = pd.DataFrame()

dropdown1 = Dropdown(options=issuer, description='Issuer:')
dropdown2 = Dropdown(options=options1, description='A:')
dropdown3 = Dropdown(options=options1, description='B:')
dropdown4 = Dropdown(options=options1, description='C:')
dropdown5 = Dropdown(options=options1, description='D:')
dropdown6 = Dropdown(options=options1, description='E:')
dropdown7 = Dropdown(options=options1, description='F:')
dropdown13 = Dropdown(options=time, description='Time:')

row1 = HBox([dropdown1, dropdown13])
row2 = HBox([dropdown2, dropdown3, dropdown4])
row3 = HBox([dropdown5, dropdown6, dropdown7])

dropdown_box = VBox([row1, row2,row3])
display(dropdown_box)

expression_input1 = Text(description='Horizontal Axis:')
expression_input2 = Text(description='Vertical Axis:')
display(expression_input1)
display(expression_input2)

def get_values():
    return dropdown1.value, dropdown2.value, dropdown3.value, \
    dropdown4.value, dropdown5.value, dropdown6.value, \
    dropdown7.value, dropdown13.value

submit_button = Button(description="Submit")
display(submit_button)

output = Output()
display(output)

def on_button_clicked(b):  
    with output:
        global df
        global df1
        global df2
        clear_output()
        
        selected_values = get_values()
        x = list(selected_values)
        selected_values = tuple([x[0]] + [f"{x[0]} {item}" for item in x[1:-1]] + [x[-1]])
        
        expression1 = expression_input1.value.upper()
        expression2 = expression_input2.value.upper()
        
        col_map = {'A': selected_values[1], 'B': selected_values[2], 'C': selected_values[3], 
                   'D': selected_values[4], 'E': selected_values[5], 'F': selected_values[6],
                  }
        
        for key, value in col_map.items():
            expression1 = expression1.replace(key, key.lower())
            expression2 = expression2.replace(key, key.lower())
        
        a, b, c, d, e, f = sp.symbols('a b c d e f')
        expr1 = sp.sympify(expression1)
        expr11 = sp.sympify(expression1)
        expr2 = sp.sympify(expression2) if expression2 else None
        expr21 = sp.sympify(expression2) if expression2 else None

        cols_to_check = [col_map['A'], col_map['B'], col_map['C'], \
                         col_map['D'], col_map['E'], col_map['F'], \
                         #col_map['G'], col_map['H'], col_map['I'], \
                         #col_map['J'], col_map['K'], col_map['L'] 
                        ]
        
        alphabets1 = re.findall(r'[a-zA-Z]', expression1)
        alphabets2 = re.findall(r'[a-zA-Z]', expression2)
        alphabets = list(sorted(set(alphabets1 + alphabets2)))
        filtered_alphabets = [char for char in alphabets if char in 'abcdef']
        upper_case_alphabets = [char.upper() for char in filtered_alphabets]
        cols_to_check = [col_map[char] for char in upper_case_alphabets]
        
        df1 = df.copy()
        df2 = df.copy()
        df1.dropna(subset=cols_to_check,inplace=True)

        replacements = {
            'A': selected_values[1], 'a': selected_values[1],
            'B': selected_values[2], 'b': selected_values[2],
            'C': selected_values[3], 'c': selected_values[3],
            'D': selected_values[4], 'd': selected_values[4],
            'E': selected_values[5], 'e': selected_values[5],
            'F': selected_values[6], 'f': selected_values[6],
            # 'G': selected_values[6], 'g': selected_values[6],
            # 'H': selected_values[7], 'h': selected_values[7],
            # 'I': selected_values[8], 'i': selected_values[8],
            # 'J': selected_values[9], 'j': selected_values[9],
            # 'K': selected_values[10], 'k': selected_values[10],
            # 'L': selected_values[11], 'l': selected_values[11]
        }

        regex = re.compile("|".join(re.escape(key) for key in replacements.keys()))
        expr111 = regex.sub(lambda match: replacements[match.group(0)],str(expr11))
        expr211 = regex.sub(lambda match: replacements[match.group(0)],str(expr21))

        if any('FWD' in v for v in col_map.values()) and not (('FWD' in expr111) or ('FWD' in expr211)):
            print("Data will be truncated as FWD is not used. Please check dropdowns.")
        else:
            df1['Expression 1'] = df1.apply(lambda row: expr1.evalf(subs={\
                a: row[col_map['A']], b: row[col_map['B']], 
                c: row[col_map['C']], d: row[col_map['D']], 
                e: row[col_map['E']], f: row[col_map['F']],
                # g: row[col_map['G']], h: row[col_map['H']],
                # i: row[col_map['I']], j: row[col_map['J']], 
                # k: row[col_map['K']], l: row[col_map['L']] 
            }), axis=1).astype(float)       
            df1['Expression 2'] = df1.apply(lambda row: expr2.evalf(subs={\
                a: row[col_map['A']], b: row[col_map['B']], 
                c: row[col_map['C']], d: row[col_map['D']], 
                e: row[col_map['E']], f: row[col_map['F']],
                # g: row[col_map['G']], h: row[col_map['H']],
                # i: row[col_map['I']], j: row[col_map['J']], 
                # k: row[col_map['K']], l: row[col_map['L']] 
            }) if expr2 else np.nan, axis=1).astype(float)
    
            plot_exp1 = np.nan
            plot_exp2 = np.nan
            
            if ('FWD' in expr111) and not ('FWD' in expr211) and expression2:
                df2['Expression 2'] = df2.apply(lambda row: expr2.evalf(subs={\
                a: row[col_map['A']], b: row[col_map['B']], 
                c: row[col_map['C']], d: row[col_map['D']], 
                e: row[col_map['E']], f: row[col_map['F']],
                # g: row[col_map['G']], h: row[col_map['H']],
                # i: row[col_map['I']], j: row[col_map['J']], 
                # k: row[col_map['K']], l: row[col_map['L']] 
                }), axis=1).astype(float)
                plot_exp2 = df2.iloc[-1]['Expression 2'].astype(float)
            if ('FWD' in expr211) and not ('FWD' in expr111):
                df2['Expression 1'] = df2.apply(lambda row: expr1.evalf(subs={\
                a: row[col_map['A']], b: row[col_map['B']], 
                c: row[col_map['C']], d: row[col_map['D']], 
                e: row[col_map['E']], f: row[col_map['F']],
                # g: row[col_map['G']], h: row[col_map['H']],
                # i: row[col_map['I']], j: row[col_map['J']], 
                # k: row[col_map['K']], l: row[col_map['L']] 
                }), axis=1).astype(float)
                plot_exp1 = df2.iloc[-1]['Expression 1'].astype(float)
            
            def filter_df(dfx1, time_period):
                dfx=dfx1.copy()
                dfx.index=pd.to_datetime(dfx.index)
                if time_period == 'All':
                    return dfx
                else:
                    now = datetime.now()
                    if time_period == '2Y':
                        start_date = now - timedelta(days=2*365)
                    elif time_period == '1Y':
                        start_date = now - timedelta(days=365)
                    elif time_period == '6M':
                        start_date = now - timedelta(days=6*30)
                    elif time_period == '3M':
                        start_date = now - timedelta(days=3*30)
                    return dfx[dfx.index.date >= start_date.date()]    
    
            def update_plot(plot_exp1,plot_exp2):
                if expression2:
                    fig = make_subplots(rows=2, cols=1, shared_xaxes=False, \
                                        vertical_spacing=0.1, specs=[ [{"secondary_y":False}], [{"secondary_y":True}] ],\
                                       subplot_titles=(f'Scatter Plot from {df1.index[0].date()} to {df1.index[-1].date()}',\
                                                      f'{expr111}: {df1[df1.columns[-2]][-1]:.4f}   &   {expr211}: {df1[df1.columns[-1]][-1]:.4f}'))
                    
                    x_scatter = df1[df1.columns[-2]]
                    y_scatter = df1[df1.columns[-1]]
        
                    coeffs = np.polyfit(x_scatter,y_scatter,2)
                    polynomial = np.poly1d(coeffs)
                    x_poly=np.linspace(min(x_scatter),max(x_scatter),500)
                    y_poly=polynomial(x_poly)
                    y_pred = polynomial(x_scatter)
                    r2=r2_score(y_scatter,y_pred)
                    equation_text = f"y = {coeffs[0]:.4f}*x² + {coeffs[1]:.4f}*x + {coeffs[2]:.4f}"
        
                    residuals = y_scatter-y_pred
                    res_sum=np.sum(residuals**2)
                    dof=len(x_scatter)-len(coeffs)
                    res_var = res_sum / dof
                    se = np.sqrt(res_var)
                    conf_upper = y_poly + 2 * se
                    conf_lower = y_poly - 2 * se
                    
                    fig.add_trace(go.Scatter(x=df1[df1.columns[-2]][:-63], y=df1[df1.columns[-1]][:-63], \
                                             name='> 3 months', mode='markers',marker=dict(color='gray')), \
                                  row=1,col=1,secondary_y=False)
                    fig.add_trace(go.Scatter(x=df1[df1.columns[-2]][-63:-1], y=df1[df1.columns[-1]][-63:-1], \
                                             name='< 3 months', mode='markers',marker=dict(color='green')), \
                                  row=1,col=1,secondary_y=False)
                    fig.add_trace(go.Scatter(x=[df1[df1.columns[-2]].iloc[-1]], y=[df1[df1.columns[-1]].iloc[-1]], \
                                             name=f'Value as of {df1.index[-1].date()}', mode='markers',marker=dict(color='red',size=10),\
                                             showlegend=True), row=1,col=1,secondary_y=False)      
                    fig.add_trace(go.Scatter(x=x_poly, y=y_poly, \
                                             name=f'Polynomial Fit<br>R² = {r2:.3f}<br>{equation_text}', \
                                             mode='lines', line=dict(color='orange'),
                                             showlegend=True), row=1,col=1,secondary_y=False) 
                   
                    fig.add_trace(go.Scatter(x=x_poly, y=conf_upper, \
                                             name='Upper Confidence Band', \
                                             mode='lines', line=dict(color='orange',dash='dash'),
                                             showlegend=True), row=1,col=1,secondary_y=False) 
                    fig.add_trace(go.Scatter(x=x_poly, y=conf_lower, \
                                             name='Lower Confidence Band', fill='tonexty',\
                                             fillcolor='rgba(255,165,0,0.15)',
                                             mode='lines', line=dict(color='orange',dash='dash'),
                                             showlegend=True), row=1,col=1,secondary_y=False) 
                    
                    if ('FWD' in expr111) and not ('FWD' in expr211):
                        plot_exp2 = [plot_exp2] * len(df1[df1.columns[-2]])
                        fig.add_trace(go.Scatter(x=df1[df1.columns[-2]], y=plot_exp2, mode='lines',\
                         line=dict(color='red',width=1),name = f'{expr211} on {df2.index[-1]} was {plot_exp2[0]}'), \
                          row=1,col=1,secondary_y=False)
    
                    if ('FWD' in expr211) and not ('FWD' in expr111):
                        plot_exp1 = [plot_exp1] * len(df1[df1.columns[-1]])
                        fig.add_trace(go.Scatter(x=plot_exp1, y=df1[df1.columns[-1]], mode='lines',\
                         line=dict(color='red',width=1), name = f'{expr111} on {df2.index[-1]} was {plot_exp1[0]}'), \
                          row=1,col=1,secondary_y=False)
                    
                    fig.add_trace(go.Scatter(x=df1.index, y=df1[df1.columns[-2]], name=expr111,\
                             line=dict(color='blue')), row=2,col=1,secondary_y=False)
                    
                    fig.add_trace(go.Scatter(x=[df1.index[0], df1.index[-1]], \
                     y=[df1[df1.columns[-2]][-1], df1[df1.columns[-2]][-1]], \
                     mode='lines', line=dict(dash='dash', color='blue'), \
                     name=f'Recent value of {expr111} is {df1[df1.columns[-2]][-1]}'),row=2,col=1,secondary_y=False)
                    
                    fig.add_trace(go.Scatter(x=df1.index, y=df1[df1.columns[-1]], \
                                         name=expr211, line=dict(color='green')),\
                                          row=2,col=1, secondary_y=True)
                    
                    fig.add_trace(go.Scatter(x=[df1.index[0], df1.index[-1]], \
                     y=[df1[df1.columns[-1]][-1], df1[df1.columns[-1]][-1]], \
                     mode='lines', line=dict(dash='dash', color='green'), \
                     name=f'Recent value of {expr211} is {df1[df1.columns[-1]][-1]}'),row=2,col=1, secondary_y=True)
                    
                    fig.update_layout(
                    hovermode='x unified',
                    plot_bgcolor='white',
                    paper_bgcolor='white',
                    legend=dict(orientation="h",yanchor="bottom",xanchor="center",x=0.5,y=1.05),
                    height=1900,
                    width=1100)
    
                else:
                    fig = make_subplots(rows=1, cols=1, shared_xaxes=False, \
                                        vertical_spacing=0.1, subplot_titles=(f'{expr111}: {df1[df1.columns[-2]][-1]:.4f}'))
                    
                    fig.add_trace(go.Scatter(x=df1.index, y=df1[df1.columns[-2]], name=expr111,\
                             line=dict(color='blue')), row=1,col=1)
                    
                    fig.add_trace(go.Scatter(x=[df1.index[0], df1.index[-1]], \
                     y=[df1[df1.columns[-2]][-1], df1[df1.columns[-2]][-1]], \
                     mode='lines', line=dict(dash='dash', color='blue'), \
                     name=f'Recent value of {expr111} is {df1[df1.columns[-2]][-1]}'))
    
                    fig.update_layout(
                    hovermode='x unified',
                    plot_bgcolor='white',
                    paper_bgcolor='white',
                    legend=dict(orientation="h",yanchor="bottom",xanchor="center",x=0.5,y=1.05),
                    height=800,
                    width=1100)
    
                def split_label(label,n):
                    return '<br>'.join([label[i:i+n] for i in range(0,len(label),n)])
    
                label1 = split_label(expr111,38)
                label2 = split_label(expr211,38)
              
                if expression2:
                    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True,\
                                     row=1,col=1,title=label1, showgrid=True, gridcolor='LightGrey',gridwidth=1)
                    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True,\
                                     row=1,col=1, title=label2, secondary_y=False, showgrid=True, gridcolor='LightGrey',gridwidth=1)
                    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True, \
                                     title='Date', row=2, col=1, showgrid=True, gridcolor='LightGrey',gridwidth=1)
                    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True, \
                                     title=label1, side='left', row=2,col=1, secondary_y=False, \
                                     showgrid=True, gridcolor='LightGrey',gridwidth=1, title_font = dict(color='blue'))
                    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True, \
                                     title=label2, side='right', row=2,col=1, secondary_y=True, title_font = dict(color='green'))
                else:
                    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True, \
                                     title='Date', row=1, col=1, showgrid=True, gridcolor='LightGrey',gridwidth=1)
                    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True, \
                                     title=label1, side='left', row=1,col=1, secondary_y=False, showgrid=True, gridcolor='LightGrey',gridwidth=1)
                fig.show()
                
            df1 = filter_df(df1, selected_values[-1]).copy() #Time goes here
            interact(update_plot(plot_exp1,plot_exp2))

submit_button.on_click(on_button_clicked)

def update_dropdowns(*args):
    selected_issuer = dropdown1.value
    if selected_issuer:
        for i in range(2, 8):
            dropdown = globals().get(f'dropdown{i}')
            if dropdown:
                x = list([item.split(' ', 1)[1] for item in options if selected_issuer in item])
                # fwd = [item for item in x if 'FWD' in item]
                # non_fwd = [item for item in x if not('FWD' in item)]
                dropdown.options = x#non_fwd + fwd

dropdown1.observe(update_dropdowns, names='value')
update_dropdowns()

Text(value='', description='Horizontal Axis:')

Text(value='', description='Vertical Axis:')

Button(description='Submit', style=ButtonStyle())

Output()